# nuScenes Map Extension Tutorial

This is the tutorial for the nuScenes map extension. In particular, the MapGraph data class. 

This tutorial will go through the description of each layers, how we retrieve and query a certain record within the map graph layers, render methods, and advanced data exploration

In database terms, layers are basically tables of the map database in which we assign arbitrary parts of the maps with informative labels such as `traffic_lights`, `stop_lines`, `road_segments`, etc. Refer to the discussion on layers for more details.

## Setup
To install the map extension, please download the files from https://www.nuscenes.org/download and copy the files into your nuScenes map folder, e.g. `/data/sets/nuscenes/maps`.

## Initialization

We will be working with the `singapore-onenorth` map. The `MapGraph` can be initialized as follows:

In [1]:
from nuscenes.map_expansion.nuscenes_map import MapGraph
import matplotlib.pyplot as plt 

map_graph = MapGraph(dataroot='/data/sets/nuscenes', map_name='singapore-onenorth')

ModuleNotFoundError: No module named 'nuscenes.map_expansion'

## Layers

Let us look at the map layers that this data class holds:

In [ ]:
map_graph.layer_names

Our map database consists of multiple **layers**. Where each layer is made up of **records**. Each record will have a token identifier.

We see how our map layers are divided into two types of layers. One set of layer belong to the `geometric_layers` group, another set of layers belongs to the `non_geometric_layers` group.  
1. `geometric_layers` serve as physical "descriptors" of the maps:
    - Nodes are the most primitive geometric records.
    - Lines consist of two or more nodes. Formally, one `Line` record can consist of more than one line segment.
    - Polygons consist of three or more nodes. A polygon can have holes, thus distorting its formal definition. Holes are defined as a sequence of nodes that forms the perimeter of the polygonal hole.
    
    
2. `non_geometric_layers` are overlapping layers that serve as labels of the physical entities. They can have more than one geometric representation (such as `drivable_areas`) but must be strictly of one type. (e.g. `road_segment` is represented by one polygon object, `lane_divider` is represented by one line object)

### 1. Geometric layers

In [ ]:
map_graph.geometric_layers

#### a. Node
The most primitive geometric record in our map database. This is the only layer that has an explicit coordinate field associated with it.

In [ ]:
sample_node = map_graph.node[0]
sample_node

#### b. Line

Similar to `Polygon` discussed above. The definition of `Line` here does not follow the formal definition as it can consist of *more than two* nodes.

In [ ]:
sample_line = map_graph.line[2]
sample_line

#### c. Polygon 
The definition of "Polygon" here does not conform to the formal definition of polygons which is a 'plane figure bounded by a closed curve' as it may contain holes.

Every polygon record comprises of a list of exterior nodes, and zero or more list(s) of nodes that constitutes (zero or more) holes.

Let's look at one polygon record

In [ ]:
sample_polygon = map_graph.polygon[3]
sample_polygon.keys()

In [ ]:
sample_polygon['exterior_node_tokens']

In [ ]:
sample_holes = sample_polygon['holes'][0]
sample_holes

### 2. Non geometric layers

Every non geometric layers are associated with an geometric object. To reiterate, the concept of "non-geometric" here does not mean that a layer does not possess any geometric entity. In fact, every non-geometric layer is associated with at least one geometric entity.

In [ ]:
map_graph.non_geometric_layers

#### a. Drivable Area
Drivable area is defined as the area where the car can drive, without consideration for driving direction or legal restrictions. This is the only layer in which the record can be represented by more than one geometric entity.

In [ ]:
sample_drivable_area = map_graph.drivable_area[0]
sample_drivable_area

In [ ]:
fig, ax = map_graph.render_record('drivable_area', sample_drivable_area['token'])

#### b. Road Segment

A segment of road on a drivable area. It has an `is_intersection` flag which denotes whether a particular road segment is an intersection.

It may or may not have an association with a `drivable area` record from its `drivable_area_token` field.

In [ ]:
sample_road_segment = map_graph.road_segment[600]
sample_road_segment

As observed, for all non geometric objects except `drivable_area`, we provide a shortcut to its `nodes`.

In [ ]:
fig, ax = map_graph.render_record('road_segment', sample_road_segment['token'])

Let's take a look at a `road_segment` record with `is_intersection == True`

In [ ]:
sample_intersection_road_segment = map_graph.road_segment[3]
sample_intersection_road_segment

In [ ]:
fig, ax = map_graph.render_record('road_segment', sample_intersection_road_segment['token'])

#### c. Road Block
Road blocks are always within a road segment. It will always have the same traffic direction within its area.

Within a road block, the number of lanes is consistent.

In [ ]:
sample_road_block = map_graph.road_block[0]
sample_road_block

Every road block has a `from_edge_line_token` and `to_edge_line_token` that denotes its traffic direction.

#### d. Lanes

Lanes are parts of the road that are designed for a single line of vehicle path.

In [ ]:
sample_lane_record = map_graph.lane[600]
sample_lane_record

In [ ]:
fig, ax = map_graph.render_record('lane', sample_lane_record['token'])

Aside from the token and the geometric representation, it has several fields:
- `lane_type` denotes whether cars or bikes are allowed to navigate through that lane.
- `from_edge_line_token` and `to_edge_line_token` denotes their traffic direction
- `left_lane_divider_segments` and `right_lane_divider_segment` denotes their lane dividers.
- `left_lane_divider_segment_nodes` and `right_lane_divider_segment_nodes` denotes the nodes that makes up the lane dividers.

#### e. Pedestrian Crossing
It is the physical world's pedestrian crossing. Each pedestrian crossing record has to be on a road segment. It has the `road_segment_token` field which denotes the `road_segment` record it is associated with.

In [ ]:
sample_ped_crossing_record = map_graph.ped_crossing[0]
sample_ped_crossing_record

In [ ]:
fig, ax = map_graph.render_record('ped_crossing', sample_ped_crossing_record['token'])

#### f. Walkway
It is the physical world's walk way or side walk.

In [ ]:
sample_walkway_record = map_graph.walkway[0]
sample_walkway_record

In [ ]:
fig, ax = map_graph.render_record('walkway', sample_walkway_record['token'])

#### g. Stop Line
The physical world's stop line, even though the name implies that it should possess a `line` geometric representation, in reality its physical representation is an **area where the ego vehicle must stop.**

In [ ]:
sample_stop_line_record = map_graph.stop_line[1]
sample_stop_line_record

It has several trivial attributes
- `stop_line_type`, the type of the stop line, this represents the reasons why the ego vehicle would stop         
- `ped_crossing_tokens` denotes the association information if the `stop_line_type` is `PED_CROSSING`.
- `traffic_light_tokens` denotes the association information if the `stop_line_type` is `TRAFFIC_LIGHT`.
- `road_block_token` denotes the association information to a `road_block`, can be empty by default. 
- `cues` field contains the reason on why this this record is a `stop_line`. An area can be a stop line due to multiple reasons:
    - Cues for `stop_line_type` of "PED_CROSSING" or "TURN_STOP" are `ped_crossing` records.
    - Cues for `stop_line_type` of TRAFFIC_LIGHT" are `traffic_light` records.
    - No cues for `stop_line_type` of "STOP_SIGN" or "YIELD".

In [ ]:
fig, ax = map_graph.render_record('stop_line', sample_stop_line_record['token'])

#### h. Carpark Area
A car park or a parking lot area.

In [ ]:
sample_carpark_area_record = map_graph.carpark_area[1]
sample_carpark_area_record

It has several trivial attributes:
- `orientation` denotes the direction of cars parked in radians.
- `road_block_token` denotes the association information to a `road_block`.

In [ ]:
fig, ax = map_graph.render_record('carpark_area', sample_carpark_area_record['token'])

#### i. Road Divider
A divider that separates one road block from another.

In [ ]:
sample_road_divider_record = map_graph.road_divider[0]
sample_road_divider_record

`road_segment_token` saves the association information to a `road_segment`.

In [ ]:
fig, ax = map_graph.render_record('road_divider', sample_road_divider_record['token'])

#### j. Lane Divider
Lane divider comes in between lanes that point in the same traffic direction.

In [ ]:
sample_lane_divider_record = map_graph.lane_divider[0]
sample_lane_divider_record

The `lane_divider_segments` field consist of different `node`s and their respective `segment_type`s which denotes their physical appearance.

In [ ]:
fig, ax = map_graph.render_record('lane_divider', sample_lane_divider_record['token'])

#### l. Traffic Light
A physical world's traffic light.

In [ ]:
sample_traffic_light_record = map_graph.traffic_light[0]
sample_traffic_light_record

It has several trivial attributes:
1. `traffic_light_type` denotes whether the traffic light is oriented horizontally or vertically.
2. `from_road_block_tokens` denotes from which road block the traffic light guides.
3. `items` are the bulbs for that traffic light.
4. `pose` denotes the pose of the traffic light.

Let's examine the `items` field

In [ ]:
sample_traffic_light_record['items']

As mentioned, every entry in the `items` field is a traffic light bulb. It has the `color` information, the `shape` information, `rel_pos` which is the relative position, and the `to_road_block_tokens` that denotes to which road blocks the traffic light bulb is guiding.

In [ ]:
map_graph.json_obj['lane'][0]

## Basic Render Methods

### Rendering multiple layers

The `MapGraph` class makes it possible to render multiple map layers on a matplotlib figure.

In [ ]:
fig, ax = map_graph.render_layers(map_graph.non_geometric_layers, figsize=(15,15))

### Rendering a particular record of the map layer

We can render a record, which will show its global and local view

In [ ]:
fig, ax = map_graph.render_record('road_segment', map_graph.road_segment[600]['token'])

In [ ]:
fig, ax = map_graph.render_record('stop_line', map_graph.stop_line[14]['token'])

## Advanced Data Exploration

Let's render a particular patch on the map:

In [ ]:
patch = (300, 1000, 500, 1200)
fig, ax = map_graph.render_map_patch(patch, map_graph.non_geometric_layers, figsize=(10, 10))

A lot of layers can be seen in this patch. Lets retrieve all map records that are in this patch.

- The option "`within`" will return all non geometric records that ***are within*** the map patch
- The option "`intersect`" will return all non geometric records that ***intersect*** the map patch


In [ ]:
records_within_patch = map_graph.get_records_in_patch(patch, map_graph.non_geometric_layers, mode='within')
records_intersect_patch = map_graph.get_records_in_patch(patch, map_graph.non_geometric_layers, mode='intersect')
print('Found %d records within the patch and %d records that intersect it.' % (len(records_within_patch), len(records_intersect_patch)))

We see that using the option `intersect` yields at least as many records as `within`.

In [ ]:
records_within_patch

In [ ]:
records_intersect_patch

Let's check what are the layers that are on point `(740, 960)`

In [ ]:
map_graph.layers_on_point(740, 960)

Looking at the above plot. Point `(760, 925)` seems to be on a stop line. Lets verify that.

In [ ]:
map_graph.record_on_point(760, 925, 'stop_line')

Let's look at the bounds/extremities of that record

In [ ]:
map_graph.get_bounds('stop_line', 'ac0a935f-99af-4dd4-95e3-71c92a5e58b1')